In [ ]:
from kth_dataset import KTH_Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from LSTMscratch import LSTMscratch
import torch
import numpy as np
import torch.nn as nn
from Model_Wrapper import Wrapper
import tqdm
import helper 


In [ ]:
helper.set_random_seed()


In [ ]:
# Create a dataset and a DataLoader
transforms = {"train":transforms.Compose([
        transforms.ToTensor(),  # Convert image to tensor

        transforms.Normalize((0.5,), (0.5,)),  # Normalize to [-1, 1]
    ]),
        "val": transforms.Compose([
        transforms.ToTensor(),  # Convert image to tensor
        transforms.Normalize((0.5,), (0.5,)),  # Normalize to [-1, 1]
    ]) }
# Initialize dataset
test_dataset = KTH_Dataset(root_dir="/home/nfs/inf6/data/datasets/kth_actions/processed", sequence_length=15,split="test", transform=transforms["train"],use_saved_samples=True)
train_dataset = KTH_Dataset(root_dir="/home/nfs/inf6/data/datasets/kth_actions/processed", sequence_length=15,split="train", transform=transforms["val"],use_saved_samples=True)


In [55]:
train_dataset.__len__()

181826

In [56]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,num_workers=8,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False,num_workers=8,pin_memory=True)
print("test_dataset per epoch",next(iter(test_loader))[0].size())

test_dataset per epoch torch.Size([128, 15, 1, 64, 64])


In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [58]:
model = LSTMscratch(input_dim=64, hidden_dim=16, number_of_layers=3, device=device)

In [59]:
# model_t = SequentialClassifierWithCells(emb_dim=128, hidden_dim=128, num_layers=2, mode="zeros")
model = model.to(device)


In [60]:
# classification loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [61]:
device

device(type='cuda')

In [62]:
model = model.to(device)

In [ ]:
classes = ["Boxing","HandClapping","HandWaving","Jogging","Running","Walking"]
# model = SequentialClassifierWithCells(emb_dim=128, hidden_dim=128, num_layers=2, mode="zeros").to(device)
writer = helper.new_writer("models","LSTMscratch")
train = Wrapper(model = model, device = device, criterion = criterion, optimizer = optimizer,writer=writer,show_progress_bar= True)
train.train(10,train_loader,test_loader,classes)
writer.close()
train.valid_accuracy()
train.plot_loss_acc()


  0%|          | 0/1421 [00:00<?, ?it/s]

Epoch 1 Iter 1421: loss 1.65925.: 100%|██████████| 1421/1421 [04:43<00:00,  5.02it/s]

Accuracy/train 24.253952538100933
Loss/train 1.744992083348832


Accuracy/test 33.300168837433674
Loss/test 1.6976338316971356


Epoch 2 Iter 1271: loss 1.63030.:  90%|█████████ | 1279/1421 [04:23<00:20,  7.01it/s]